# Brazilian Investment Funds Overview

This project main objective is to map investment fund market in Brazil. Investment funds can trade in different types of assets, like financial products (stocks, options, debentures), real state or buying part of other funds. Each fund raises capital by offering quotas, this can be an ongoing process (open fund) or only one time offer (closed fund).

The funds are created and managed by a financial institution (manager). Managers will also take care of all the paper work involving fund rules, which index should investors use as benchmark and general structure so a bank can issue the fund. The bank role is so ensure the fund is legit, has the paperwork in order and distribute the financial sheets so potential investor can buy a quota. This is a very efficient way to poll resources and diversify the risk for all parties involved.

Learn more in ANBIMA [PDF](https://www.anbima.com.br/data/files/D7/B6/AD/5E/369EC8104606BDC8B82BA2A8/CPA-10-Cap5.pdf).

Now that we know what is an investment fund, and how is its structure, let's be curious about it:
- How many funds are in the market?
- What are the main assets traded? Financial? Real State? Credit?
- Who are the big managers?
- Who are the main issuers?
- How was the market for the last years?

## Importing libraries

In [1]:
import pandas as pd # manipulating data
import numpy as np  # basic math operations
import matplotlib.pyplot as plt # graphs
import seaborn as sns   #graphs
import requests # request files on Brazilian Securities and Exchange Comission (CVM)
import zipfile  # unzip CVM files
import os   # manipulate disk files

## Downloading data
Investment Funds must be registered in Brazilian Securities and Exchange Comission (CVM). CVM has funds data on their daily returns, benchmark index, type of fund, issuer, manager and other informations related to it. All data is open to the public in CVM [website](https://dados.cvm.gov.br/group/fundos-de-investimento):
- Funds return: one database for daily, monthly, quarterly, anual
- Register info: funds name, manager, issuer, type, open/close
- Statement of Income: database with funds link to their state of income
- Investors profile: who owns funds quotas (other business, retirement funds, individual investors, professional investors)
- Performance metrics: how to calculate return, risk accordin to managers, collateral

For this study, I'm interested in the daily returns and register info. This way I can map funds by their features and follow their performance in time. The advantage to use daily data is that I can transform daily info into month, quarter and annual.

In [2]:
# Creating parameters to download data
## Date paramenters to match CVM files
years = ['2024','2023','2022','2021','2020']    # Creating a five year window so we can see the end of pandemic and current government
legacy = ['2020']   # Creating legacy list, CVM moves old data to another URL/directory

months = range(1,13)    # Crete month list from Jan(01) to Dec(12)
month_list = []     # List must be a string bc I'll add each emelento to a url request to CVM

for i in months:    # Transform each integer element into a string element
    if i<10:    # For months with only one digit, we need to add zero (0) before to match the csv file
        i = str('0'+str(i))
    else:   # Months with two digits only need to be converted to string
        i = str(i)
    month_list.append(i)    # Append each string to the month list

To collect the data, I'll request directly from CVM website the zipfiles containing the [daily returns of financial funds](https://dados.cvm.gov.br/dataset/fi-doc-inf_diario). The main issue here, is that we have two types of repositories: (1) monthly zip files from the current year to 3 years ago (Y-3), (2) yearly zip file for older data. So in my study 2024,2023,2022 and 2021 will have monthly zip files, while [2020](https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/HIST/) will have only one zip file with 12 csv files inside.

To solve it, I'll just need to create a loop to identify which year is considered old/legacy by CVM. Previously I created a legacy list with 2020 so I can use it in the loop now.

URL with daily return data
URL model: dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/**DADOS/inf_diario_fi_202307**.zip
- replace the date on the URL '202307' with the loop

URL for legacy data
URL legacy model: dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/**DADOS/HIST/inf_diario_fi_2000**.zip
- replace only the year 2000

In [3]:
cvm_daily_return = pd.DataFrame()   # Create an empty dataframe for daily returns
cvm_legacy_return = pd.DataFrame()  # Creawte an empty dataframe for legacy data

# Create loop to download data from 2020 to July 2024
## I'll collect data for each year in our years list
for yyyy in years:
    try:
        if yyyy in legacy:  # Y-3 data is considered history and moved to a different directory, I'll call it legacy
            daily_return_url = f'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/HIST/inf_diario_fi_{yyyy}.zip'
            download_url = requests.get(daily_return_url)   # Access URL with zipfile, else would be just a string on our local machine
            zip_filename = f'inf_diario_fi_{yyyy}.zip'  # Save zipfile name, only asks for year to identify it. All 12 month csv are inside it.
            with open(zip_filename,'wb') as zip_ref:    # Open zipfile from CVM url. Using 'wb' because I need to Write ('w') the file in Binary('b')
                zip_ref.write(download_url.content)     # Write/save zipfile on local disk. I used '.content' to write/save the files withing the URL zip
            with zipfile.ZipFile(zip_filename, 'r') as cvm_zip: # Legacy Zip has 12 csv files, so I'll just read and concatenate all
                legacy_csv = [pd.read_csv(cvm_zip.open(f), sep=';') for f in cvm_zip.namelist()]
                cvm_legacy_return = pd.concat(legacy_csv)
            os.remove(zip_filename)  # Delete zipfile from disk so I can keep a clean directory
        else:
            for mm in month_list:   # Y-3< data is called by year and month. So we need to run all month_list elements
                daily_return_url = f'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{yyyy}{mm}.zip'
                download_url = requests.get(daily_return_url)   # Access URL with zipfile
                zip_filename = f'inf_diario_fi_{yyyy}{mm}.zip'  # Save zipfile name, it's important to name it here bc I'll delete the zipfile after
                with open(zip_filename, 'wb') as zip_ref:   # Open zipfile from CVM url
                    zip_ref.write(download_url.content)     # Write/save zipfile on local disk. I used '.content' to write/save the files withing the URL zip
                with zipfile.ZipFile(zip_filename) as cvm_zip:  # Manipulate recent saved zipfile on our disk
                    for file_name in cvm_zip.namelist():    # Look for all files within zipfile
                        if file_name.endswith('.csv'):      # If a file ends with '.csv' I'll open and read it with pandas
                            with cvm_zip.open(file_name) as cvm_csv:    # I'll create a temporary obj, so we can concatenate it with our main dataframe
                                cvm_daily_return_temp = pd.read_csv(cvm_csv, sep=';')   # Read csv, but Brazilian data is separated with ';'
                                cvm_daily_return = pd.concat([cvm_daily_return, cvm_daily_return_temp]) # Concat main dataframe with temporary
                os.remove(zip_filename)  # Delete zipfile from disk so I can keep a clean directory
        cvm_daily_return = pd.concat([cvm_daily_return,cvm_legacy_return])  # Concatenate current and legacy data
    except:
        pass    # Avoid stopping the process in case it looks for current year and months yet to come (YYYYMM+1)

cvm_daily_return    # Observe we get all monthly data from 2024 to 2000

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,FI,00.017.024/0001-53,2024-01-02,1136699.13,34.298860,1139708.10,0.0,0.0,1
1,FI,00.017.024/0001-53,2024-01-03,1137245.82,34.312303,1140154.80,0.0,0.0,1
2,FI,00.017.024/0001-53,2024-01-04,1137741.93,34.326023,1140610.71,0.0,0.0,1
3,FI,00.017.024/0001-53,2024-01-05,1138240.64,34.338221,1141016.02,0.0,0.0,1
4,FI,00.017.024/0001-53,2024-01-08,1138427.98,34.350495,1141423.89,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...
422558,FI,97.929.213/0001-34,2020-12-24,70707449.04,9.302694,70694161.72,0.0,0.0,2
422559,FI,97.929.213/0001-34,2020-12-28,70931676.80,9.332148,70917993.06,0.0,0.0,2
422560,FI,97.929.213/0001-34,2020-12-29,69872876.85,9.339427,70973306.23,0.0,0.0,2
422561,FI,97.929.213/0001-34,2020-12-30,70145454.01,9.360174,71130974.38,0.0,0.0,2


The next step is to identify the funds. Investors know it by their name, not their register number. I'll also need to check their status, to see if the fund still active. Here I face a similar situation from return data, [current](https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv) and [legacy](https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi_hist.zip) data.

In [26]:
# Open current register file
cvm_register = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv', sep=';', encoding='latin-1')

# Open legacy register file
## Issue: legacy has several csvs in one zip file
register_url = 'https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi_hist.zip'    # Find legacy url
download_register = requests.get(register_url)  # Reach url online
register_zip = 'cad_fi_hist.zip'    # Create an object with zipfile name so we can use it on next steps
with open(register_zip,'wb') as reg_ref:    # Open zipfile to write it on disk
    reg_ref.write(download_register.content)
with zipfile.ZipFile(register_zip,'r') as register_zip: # Read zip file
    reg_csv_lag = [pd.read_csv(register_zip.open(g), sep=';', encoding='latin-1') for g in register_zip.namelist()] # Read each csv and put in all together in one object as a list
    legacy_register = pd.concat(reg_csv_lag, axis=0, ignore_index=True) # Read all lists into one dataframe
os.remove('cad_fi_hist.zip')    # Delete legacy refister zip file from our disk

C:\Users\anaak\AppData\Local\Temp\ipykernel_12456\3861353282.py:2: DtypeWarning: Columns (14,17,18,20,22,24,27,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  cvm_register = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv', sep=';', encoding='latin-1')


In [49]:
# Check if headers from current and legacy register dataframes are the same
## Print the shape of each dataframe to see if they are the same
if cvm_register.shape[1] > legacy_register.shape[1]:    # Analyze the shape(0_row, 1_col). We are interested in cols so shape[1]
    cur_lag = cvm_register.shape[1] - legacy_register.shape[1]
    print('Current register dataframe has',cur_lag, 'more columns')
else:
    lag_cur = legacy_register.shape[1] - cvm_register.shape[1] 
    print('Legacy register dataframe has',lag_cur,'more columns')
print('CVM register shape:',cvm_register.shape) # General idea of shape for current register dataframe
print('Legacy register shape:',legacy_register.shape)# General idea of shape for legacy register dataframe
print('\n')

# Find which columns are convergent and divergent between them
reg_cur_col = cvm_register.columns      # Get columns names for current register dataframe
reg_lag_col = legacy_register.columns   # Get columns names for legacy register dataframe

# Create an object for coluimns in common and columns in legacy and not current
common_cols = reg_cur_col.intersection(reg_lag_col)
cur_not_lag = reg_cur_col.difference(reg_lag_col)
lag_not_cur = reg_lag_col.difference(reg_cur_col)

print(common_cols.nunique(),'columns in COMMON between current and legacy:')
print(common_cols)
print('\n')
print(cur_not_lag.nunique(),'columns in CURRENT register dataframe and not in LEGACY:')
print(cur_not_lag)
print('\n')
print(lag_not_cur.nunique(),'columns in LEGACY register dataframe and not in CURRENT:')
print(lag_not_cur)

Legacy register dataframe has 23 more columns
CVM register shape: (79591, 41)
Legacy register shape: (1836426, 64)


29 columns in COMMON between current and legacy:
Index(['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_REG', 'SIT', 'DT_INI_SIT',
       'DT_INI_EXERC', 'DT_FIM_EXERC', 'CLASSE', 'DT_INI_CLASSE',
       'RENTAB_FUNDO', 'CONDOM', 'FUNDO_COTAS', 'FUNDO_EXCLUSIVO',
       'TRIB_LPRAZO', 'PUBLICO_ALVO', 'TAXA_ADM', 'INF_TAXA_ADM', 'DIRETOR',
       'CNPJ_ADMIN', 'ADMIN', 'PF_PJ_GESTOR', 'CPF_CNPJ_GESTOR', 'GESTOR',
       'CNPJ_AUDITOR', 'AUDITOR', 'CNPJ_CUSTODIANTE', 'CUSTODIANTE',
       'CNPJ_CONTROLADOR', 'CONTROLADOR'],
      dtype='object')


12 columns in CURRENT register dataframe and not in LEGACY:
Index(['CD_CVM', 'CLASSE_ANBIMA', 'DT_CANCEL', 'DT_CONST', 'DT_INI_ATIV',
       'DT_PATRIM_LIQ', 'ENTID_INVEST', 'INF_TAXA_PERFM', 'INVEST_CEMPR_EXTER',
       'TAXA_PERFM', 'TP_FUNDO', 'VL_PATRIM_LIQ'],
      dtype='object')


35 columns in LEGACY register dataframe and not in CURRE

#### Comparing dataframes
On this step I noticed a difference between columns within current and legacy register dataframe, and why is that?
- According to CVM notes, they changed the infos required from funds along the years. This can happen due a law change or CVM don't see the need to ask for that information (e.g: legacy  'DT_INI_TAXA_ADM' has the date for when administration fee was charged)
- Some headers changed their name. Due to operational reasons, CVM data managers change the name of the columns to fit their system (e.g: legacy 'VL_TAXA_PERFM' and current 'TAXA_PERFM'. Both them have the performance rate for managers based on funds gains)
- New columns addition. With the law change, CVM may ask for new information (e.g: current 'CLASSE_ANBIMA')

Based on those difference, I'll work with the columns they have in common. In total, there are **29 in common columns** I can work with, but not all data in needed.

**Columns in common between current and legacy register:**

*'CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_REG', 'SIT', 'DT_INI_SIT',
'DT_INI_EXERC', 'DT_FIM_EXERC', 'CLASSE', 'DT_INI_CLASSE',
'RENTAB_FUNDO', 'CONDOM', 'FUNDO_COTAS', 'FUNDO_EXCLUSIVO',
'TRIB_LPRAZO', 'PUBLICO_ALVO', 'TAXA_ADM', 'INF_TAXA_ADM', 'DIRETOR',
'CNPJ_ADMIN', 'ADMIN', 'PF_PJ_GESTOR', 'CPF_CNPJ_GESTOR', 'GESTOR',
'CNPJ_AUDITOR', 'AUDITOR', 'CNPJ_CUSTODIANTE', 'CUSTODIANTE',
'CNPJ_CONTROLADOR', 'CONTROLADOR'*

**The data I'll need to identify investment funds are:**

*'CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_REG', 'SIT','CLASSE', 'DT_INI_CLASSE','RENTAB_FUNDO', 'CONDOM', 'FUNDO_COTAS', 'FUNDO_EXCLUSIVO','CPF_CNPJ_GESTOR', 'GESTOR','CNPJ_AUDITOR', 'AUDITOR', 'CNPJ_CUSTODIANTE', 'CUSTODIANTE',*

Based on CVMs [dictionary](https://dados.cvm.gov.br/dados/FI/CAD/META/meta_cad_fi.txt), I'll use the following cols:
| Column | Description|
| --- | ---|
|CNPJ_FUNDO| Investment fund register code 
|DENOM_SOCIAL| Investment fund name 
|DT_REG| Register date
|SIT| Situation (Active, Deactive)
|CLASSE| Type of assets
|DT_INI_CLASSE| Date from when assets were purchased
|RENTAB_FUNDO| Rentability
|CONDOM| Open/Close fund
|FUNDO_COTAS| If fund has quotas or not
|FUNDO_EXCLUSIVO| Exclusive fund
|CPF_CNPJ_GESTOR| Manager register code
|GESTOR| Manager name
|CNPJ_AUDITOR| Audit firm register code
|AUDITOR| Audit firm name
|CNPJ_CUSTODIANTE| Issuer register code
|CUSTODIANTE| Issuer name

These columns will give me an idea of each fund structue. Those columns will tell me what type of assets each fund is working with, when they started trading those, who is the manager choosing the assets and who is issuing the quotas. And why does it matter?

I can analyze funds performance over the years and identify if a manager has better results than the others. Funds features (such as open/closed or exclusive/not) may indicate better performing funds due to private information or access to better assets. Issuer can hold a specific type of asset or only issue for a certain type of investor, and therefore have a different performance. By keeping audit firm data, I can point who are the big firms working with investment funds or a specific type of asset.

In [56]:
# Merging current and legacy register dataframes based on the columns I selected
main_cols = ['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_REG', 'SIT','CLASSE', 'DT_INI_CLASSE','RENTAB_FUNDO', 'CONDOM', 'FUNDO_COTAS', 'FUNDO_EXCLUSIVO','CPF_CNPJ_GESTOR', 'GESTOR','CNPJ_AUDITOR', 'AUDITOR', 'CNPJ_CUSTODIANTE', 'CUSTODIANTE']
cvm_complete_reg = cvm_register[main_cols].copy()   # Copu current register dataframe data
pd.concat([cvm_register,legacy_register], join='inner')   # Concat based on columns in common
cvm_complete_reg.shape  # Analyze the object shape

(79591, 16)

In [67]:
# Check data type
print(cvm_complete_reg.info())  # .info() to see how variables are stored. Observe that date variables are stored as string
# Convert 'DT_REG' and 'DT_INI_CLASSE' in datetime variables
cvm_complete_reg['DT_REG'] = pd.to_datetime(cvm_complete_reg['DT_REG'])
cvm_complete_reg['DT_INI_CLASSE'] = pd.to_datetime(cvm_complete_reg['DT_INI_CLASSE'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79591 entries, 0 to 79590
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CNPJ_FUNDO        79591 non-null  object        
 1   DENOM_SOCIAL      79591 non-null  object        
 2   DT_REG            79591 non-null  datetime64[ns]
 3   SIT               79591 non-null  object        
 4   CLASSE            66285 non-null  object        
 5   DT_INI_CLASSE     66285 non-null  object        
 6   RENTAB_FUNDO      50186 non-null  object        
 7   CONDOM            65937 non-null  object        
 8   FUNDO_COTAS       66291 non-null  object        
 9   FUNDO_EXCLUSIVO   55833 non-null  object        
 10  CPF_CNPJ_GESTOR   52445 non-null  object        
 11  GESTOR            52445 non-null  object        
 12  CNPJ_AUDITOR      51719 non-null  object        
 13  AUDITOR           51719 non-null  object        
 14  CNPJ_CUSTODIANTE  5089

In [68]:
# Check final result
print(cvm_complete_reg.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79591 entries, 0 to 79590
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CNPJ_FUNDO        79591 non-null  object        
 1   DENOM_SOCIAL      79591 non-null  object        
 2   DT_REG            79591 non-null  datetime64[ns]
 3   SIT               79591 non-null  object        
 4   CLASSE            66285 non-null  object        
 5   DT_INI_CLASSE     66285 non-null  datetime64[ns]
 6   RENTAB_FUNDO      50186 non-null  object        
 7   CONDOM            65937 non-null  object        
 8   FUNDO_COTAS       66291 non-null  object        
 9   FUNDO_EXCLUSIVO   55833 non-null  object        
 10  CPF_CNPJ_GESTOR   52445 non-null  object        
 11  GESTOR            52445 non-null  object        
 12  CNPJ_AUDITOR      51719 non-null  object        
 13  AUDITOR           51719 non-null  object        
 14  CNPJ_CUSTODIANTE  5089

In [80]:
# Check to see if I got the current data right
cvm_complete_reg.sort_values(by=['DT_REG'], inplace=True,ascending=False)
cvm_complete_reg.head()

,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,SIT,CLASSE,DT_INI_CLASSE,RENTAB_FUNDO,CONDOM,FUNDO_COTAS,FUNDO_EXCLUSIVO,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE
73742,56.211.321/0001-90,IT NOW IBOVESPA B3 BR+ FUNDO DE ÍNDICE - RESPO...,2024-08-01,FASE PRÉ-OPERACIONAL,NaN,NaT,NaN,NaN,NaN,NaN,40.430.971/0001-96,ITAU UNIBANCO ASSET MANAGEMENT LTDA.,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES...,60.701.190/0001-04,ITAU UNIBANCO S.A.
73595,56.212.023/0001-14,RIZA LECCI FUNDO DE INVESTIMENTO IMOBILIÁRIO R...,2024-08-01,FASE PRÉ-OPERACIONAL,FII,2024-08-01,NaN,Fechado,N,NaN,12.209.584/0001-99,RIZA GESTORA DE RECURSOS LTDA.,NaN,NaN,30.306.294/0001-45,BANCO BTG PACTUAL S/A
66472,56.213.199/0001-90,FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS ...,2024-08-01,FASE PRÉ-OPERACIONAL,FIDC,2024-08-01,NaN,Fechado,N,NaN,31.681.693/0001-59,INVESTCOOP ASSET MANAGEMENT LTDA.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,30.306.294/0001-45,BANCO BTG PACTUAL S/A
60922,56.209.706/0001-12,CAIXA BRASIL 2032 X II TÍTULOS PÚBLICOS FUNDO ...,2024-08-01,FASE PRÉ-OPERACIONAL,Fundo de Renda Fixa,2024-08-01,OUTROS,Aberto,N,N,42.040.639/0001-40,CAIXA DISTRIBUIDORA DE TÍTULOS E VALORES MOBIL...,49.928.567/0001-11,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENT...,00.360.305/0001-04,CAIXA ECONOMICA FEDERAL
77359,56.217.451/0001-30,DYNA FUNDO DE INVESTIMENTO EM PARTICIPAÇÕES MU...,2024-08-01,FASE PRÉ-OPERACIONAL,FIP Multi,2024-08-01,NaN,Fechado,N,NaN,09.437.736/0001-04,LORINVEST GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.,30.306.294/0001-45,BANCO BTG PACTUAL S/A


In [ ]:
# Check to see if I got the legacy data right
cvm_complete_reg.tail()
## For our tail data, we have really old data from 1950. I won't need data before 2020
cvm_complete_reg = cvm_complete_reg[cvm_complete_reg['DT_REG'].dt.year >= 2020]

In [84]:
# Check to see if I got the legacy data right
cvm_complete_reg.tail()

,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,SIT,CLASSE,DT_INI_CLASSE,RENTAB_FUNDO,CONDOM,FUNDO_COTAS,FUNDO_EXCLUSIVO,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE
72723,34.197.811/0001-46,VALORA CRI ÍNDICE DE PREÇO FUNDO DE INVESTIMEN...,2020-01-03,EM FUNCIONAMENTO NORMAL,FII,2020-01-02,NaN,Fechado,N,NaN,07.559.989/0001-17,VALORA IMOBILIÁRIO E INFRAESTRUTURA LTDA.,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES...,30.306.294/0001-45,BANCO BTG PACTUAL S/A
41079,35.088.124/0001-55,STRATFORD FI EM COTAS DE FI MULTIMERCADO CREDI...,2020-01-03,EM FUNCIONAMENTO NORMAL,Fundo Multimercado,2020-01-02,NaN,Fechado,S,S,60.451.242/0001-23,BTG PACTUAL WM GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.,30.306.294/0001-45,BANCO BTG PACTUAL S/A
36492,30.068.195/0001-72,FUNDO DE INVEST EM COTAS DE FUNDOS DE INVEST C...,2020-01-03,EM FUNCIONAMENTO NORMAL,Fundo de Renda Fixa,2018-02-07,DI de um dia,Aberto,S,N,42.040.639/0001-40,CAIXA DISTRIBUIDORA DE TÍTULOS E VALORES MOBIL...,49.928.567/0001-11,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENT...,00.360.305/0001-04,CAIXA ECONOMICA FEDERAL
41108,35.136.864/0001-10,EUQUEROINVESTIR CBI HEDGE FUNDO DE INVESTIMENT...,2020-01-02,CANCELADA,Fundo Multimercado,2019-09-11,DI de um dia,Aberto,N,N,NaN,NaN,NaN,NaN,NaN,NaN
72206,15.798.220/0001-80,ORCHID FUNDO DE INVESTIMENTO IMOBILIÁRIO,2020-01-02,EM FUNCIONAMENTO NORMAL,FII,2020-01-02,NaN,NaN,N,NaN,23.863.529/0001-34,REAG ADMINISTRADORA DE RECURSOS LTDA.,15.454.120/0001-36,MGI ASSURANCE AUDITORES INDEPENDENTES SS,NaN,NaN
